In [3]:
import csv
import re 

import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import molmass

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import HuberRegressor, LinearRegression

In [4]:
# colors and class-superclass maps
with open(r'..\data\metadata\color_schemes.json') as infile:
    colors = json.load(infile, parse_int=True)
    
ap = pd.read_excel(r'..\data\metadata\animal_phenotypes.xlsx', index_col=0)

fg = pd.read_csv(r'..\data\metadata\combined_metab_lipid_file_grouping.csv', index_col=0)

# Can't use the filtered dataset because most of the FAs have been removed
# Also don't use the Final Results output, use the Unfiltered Results 
# data = pd.read_csv('../data/processed/lipidomics_linregress_normalized_data.csv')
data = pd.read_csv(r'../data/search/CD31/peak_finder/Unfiltered_Results.csv')
data.rename({'Quant Ion': 'm/z', 'Retention Time (min)': 'RT', 'Identification': 'ID', 
             'Area (max)': 'Area (Max.)'}, inplace=True, axis=1)
data.rename({fr_name: mod_name for fr_name, mod_name in zip(fg['fr_name'], fg.index)}, inplace=True, axis=1)
data_cols = data.filter(regex='_RBG|_FBG').columns
data['mean'] = data[data_cols].mean(axis=1)
data['is_id'] = data['ID'].notna()
data.drop(data.filter(regex='Unnamed').columns, axis=1, inplace=True)

data

,RT,m/z,Polarity,Area (Max.),ID,Lipid Class,Features Found,Filter Status,20210729_AJ_Toh_RatBloodGlucose_ExtractionBlank.raw (F1),20210729_AJ_Toh_RatBloodGlucose_SolventBlank.raw (F2),...,1102_8_RBG,1102_9_FBG,1102_9_RBG,1102_10_FBG,1102_10_RBG,20210729_AJ_Toh_RatBloodGlucose_Water_R1.raw (F65),20210729_AJ_Toh_RatBloodGlucose_Water_R2.raw (F66),20210729_AJ_Toh_RatBloodGlucose_Water_R3.raw (F67),mean,is_id
0,0.528,664.07684,+,7.218670e+06,NaN,NaN,2,Dimer,3616.600742,3754.897464,...,2.456581e+06,4.949765e+06,3.271367e+06,3.898191e+06,3.586652e+06,4371.310119,3837.969412,3872.151793,3.760267e+06,False
1,0.529,662.08582,+,2.056865e+07,NaN,NaN,4,Dimer,4138.791646,4297.056647,...,1.187316e+07,1.415339e+07,5.295644e+06,9.765149e+06,1.210703e+07,5002.471406,4392.123121,4457.358249,1.230879e+07,False
2,0.530,991.11859,+,9.722884e+06,NaN,NaN,6,Adduct of existing peak,5222.040196,5421.727995,...,5.513680e+06,5.899307e+06,7.166797e+06,5.926878e+06,6.104699e+06,6484.218919,5541.676278,5623.985437,5.589505e+06,False
3,0.531,661.08264,+,7.778703e+07,NaN,NaN,44,Dimer,4970.354146,5160.417616,...,3.948893e+07,4.646823e+07,6.052133e+07,4.309350e+07,4.116755e+07,6171.699792,5274.584766,5352.926881,4.483563e+07,False
4,0.532,659.06866,-,6.691758e+07,NaN,NaN,47,Dimer,28520.354100,6130.634650,...,4.258937e+07,3.593391e+07,5.412068e+07,4.535156e+07,4.205162e+07,9011.421356,6850.520434,6319.495392,3.603261e+07,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3133,20.434,967.86713,+,3.380837e+06,NaN,NaN,4,Adduct of existing identified peak,12843.405760,13979.736960,...,7.847792e+05,2.203746e+05,1.180822e+06,3.224706e+05,5.614904e+05,23499.690490,14932.771020,18051.601980,4.950699e+05,False
3134,20.435,1003.93829,+,3.203813e+06,NaN,NaN,4,NaN,12695.823470,13819.097200,...,7.912981e+05,1.910285e+05,1.147269e+06,4.434897e+05,5.509051e+05,23229.657890,14761.180040,17844.172830,4.707392e+05,False
3135,20.452,1164.01404,+,3.204960e+06,NaN,NaN,2,NaN,13749.188890,14296.409340,...,7.236153e+05,3.054687e+04,1.097559e+06,2.851977e+04,5.293939e+05,24032.011160,15271.031760,18460.511230,2.975804e+05,False
3136,20.473,936.89600,+,2.627972e+06,TG 56:0,TG,6,RT out of class range,12059.809380,12539.792190,...,7.020718e+05,4.570013e+05,1.062921e+06,7.064462e+05,6.040003e+05,101141.455400,13090.673690,19330.809740,6.179504e+05,True


# Massage data into right form for PyMC modeling 